You can look at a process’ state by reading

    /proc/<PID>/status | grep State
    
Replace <PID> with the process ID (or self)

Results: 

- R: Running and runnable [Running and Waiting]
- S: Interruptible sleep [Blocked]
- D: Uninterruptible sleep [Blocked] (waiting for IO, network, etc., such that we cannot "wake it up" before the wait finishes)
- T: Stopped
- Z: Zombie

#### *If there are parents and child relationship between every single processes, how does the first one starts?*

> After the kernel initializes, it creates a single process from a program (pid = 1)

This process is called `init`, and it looks for it in `/sbin/init`  
Responsible for executing every other process on the machine --> This is ultimate ancestor  
Must always be active, if it exits the kernel thinks you’re shutting down

For Linux, `init` will probably be systemd but there’s other options

Aside: some operating systems create an “idle” process that the scheduler can run

![Alt text](images/image7.png)

(see: each tab is a process)


You can see your process tree with:

    htop

*(You can press F5 to switch between tree and list view)*

### Processes are assigned a Process ID (pid) on creation, and does not change throughout its lifetime

The process ID is just a number, and is unique for every **active** process  
On most Linux systems the maximum pid 32768, and <mark>0 is reserved (invalid)</mark>

Eventually the kernel will recycle a pid, after the process dies, for a new process

Remember: each process has its own address space (independent view of memory)

### *What happends if the parent process exits (dies) before the child process dies??*

*** There's a step between when the process exits and when the OS actually removes it  

--> the PCB is still kept

The operating system sets the exit status when a process terminates (the process terminates by calling exit)  
<mark>It can’t remove its PCB yet</mark>

The minimum acknowledgment the parent has to do is read the child’s exit status

There’s two situations:
1. The child exits first (zombie process)
2. The parent exits first (orphan process)

These situations will remain, before it's **acknowledged** by the parent.

How does this acknowledgement works?

--> System call `wait`

wait as the following API:
- status: Address to store the wait status of the process
- Returns the process ID of child process
    - -1: on failure
    - 0: for non blocking calls with no child changes
    - \>0: the child with a change
    
The wait status contains a bunch of information, including the exit code  
Use `man wait` to find all the macros to query wait status  
You can use `waitpid` to wait on a specific child process  



In [1]:
%%file run.c

#include <errno.h>
#include <stdio.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <unistd.h>

int main(void) {
  pid_t pid = fork();
  if (pid == -1) {
    return errno;
  }
  if (pid == 0) {
    sleep(2);
  }
  else {
    printf("Calling wait\n");
    int wstatus;
    pid_t wait_pid = wait(&wstatus);
    if (WIFEXITED(wstatus)) {
      printf("Wait returned for an exited process! pid: %d, status: %d\n", wait_pid, WEXITSTATUS(wstatus));
    }
    else {
      return ECHILD;
    }
  }
  return 0;
}

Overwriting run.c


`init` does 2 jobs only: create processes, and call waitpid (to wait for its child (the actual child/assigned orphans)) to die

In [ ]:
%%file run.c

int main(int argc, char *argv[]) {
    pid_t pid = fork();
    if (pid ⿂⿻ -1) {
        int err = errno;
        perror("fork failed");
        return err;
    }
    if (pid ⿂⿻ 0) {
        printf("Child parent pid: %d\n", getppid());        // *** This will always print the parent's pid 
        sleep(2);
        printf("Child parent pid (after sleep): %d\n", getppid());
    }
    else {
        sleep(1);   // Say the parent process runs here first, then during this 1 second, the child process will be ran anyways so (***)
    }
    return 0;
}

